<a href="https://colab.research.google.com/github/tintamaria95/CocoriGo/blob/main/importGolois_MLD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://www.lamsade.dauphine.fr/~cazenave/project2022.zip
!unzip project2022.zip

In [6]:
!python golois.py

getValidation
r.shape = (10000, 19, 19, 31)
nbExamples = 10000
tcmalloc: large alloc 2400002048 bytes == 0x5792c000 @  0x7f0a68b62887 0x7f09ee6d30d9 0x7f09ee6d885f 0x7f09ee6ed06f 0x58f6e4 0x5105e2 0x5b4ee6 0x6005a3 0x607796 0x60785c 0x60a436 0x64db82 0x64dd2e 0x7f0a6875dc87 0x5b636a
nbPositionsSGF = 29425326
nbPositionsSGF = 29425326
loading validation.data
2022-11-10 14:58:18.603762: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 board (InputLayer)             [(None, 19, 19, 31)  0           []                               
                                ]                                                                 
             

##Réentraînement de modèle

In [7]:
from tensorflow import keras
import numpy as np
import gc

import golois

epochs = 100

planes = 31
moves = 361
N = 10000
batch = 128

input_data = np.random.randint(2, size=(N, 19, 19, planes))
input_data = input_data.astype ('float32')
policy = np.random.randint(moves, size=(N,))
policy = keras.utils.to_categorical (policy)
value = np.random.randint(2, size=(N,))
value = value.astype ('float32')
end = np.random.randint(2, size=(N, 19, 19, 2))
end = end.astype ('float32')
groups = np.zeros((N, 19, 19, 1))
groups = groups.astype ('float32')
golois.getValidation (input_data, policy, value, end)

# Load models
model_name = "model_resnet_SGD_lr5_10-3.h5"
model = keras.models.load_model(model_name)
m_names = model.metrics_names

for i in range (1, epochs + 1):
    print ('epoch ' + str (i))
    golois.getBatch (input_data, policy, value, end, groups, i * N)
    history = model.fit(input_data,
                        {'policy': policy, 'value': value}, 
                        epochs=1, batch_size=batch)
    if (i % 20 == 0):
        gc.collect ()
        golois.getValidation (input_data, policy, value, end)
        val = model.evaluate (input_data,
                              [policy, value], verbose = 0, batch_size=batch)
        print ("val =", val)
        model.save (model_name)

print("")
print("*******************")
print(model_name)
for i, metric in enumerate(m_names):
  print(metric, ":", val[i])
#display(keras.utils.plot_model(model, str(model_name + ".png"), show_shapes=True))

epoch 1
79/79 [==============================] - 3s 20ms/step - loss: 4.0841 - policy_loss: 3.3855 - value_loss: 0.6893 - policy_categorical_accuracy: 0.2561 - value_mse: 0.1196
epoch 2
79/79 [==============================] - 1s 19ms/step - loss: 3.9953 - policy_loss: 3.2966 - value_loss: 0.6894 - policy_categorical_accuracy: 0.2616 - value_mse: 0.1175
epoch 3
79/79 [==============================] - 2s 19ms/step - loss: 3.9766 - policy_loss: 3.2786 - value_loss: 0.6887 - policy_categorical_accuracy: 0.2650 - value_mse: 0.1183
epoch 4
79/79 [==============================] - 2s 19ms/step - loss: 3.8664 - policy_loss: 3.1669 - value_loss: 0.6902 - policy_categorical_accuracy: 0.2858 - value_mse: 0.1193
epoch 5
79/79 [==============================] - 2s 20ms/step - loss: 3.8512 - policy_loss: 3.1513 - value_loss: 0.6906 - policy_categorical_accuracy: 0.2908 - value_mse: 0.1182
epoch 6
79/79 [==============================] - 2s 19ms/step - loss: 3.8384 - policy_loss: 3.1392 - value_los

# Comparaison de modèles

In [5]:
from tensorflow import keras
import numpy as np
import gc
import os

import golois

planes = 31
moves = 361
N = 10000
batch = 128

input_data = np.random.randint(2, size=(N, 19, 19, planes))
input_data = input_data.astype ('float32')
policy = np.random.randint(moves, size=(N,))
policy = keras.utils.to_categorical (policy)
value = np.random.randint(2, size=(N,))
value = value.astype ('float32')
end = np.random.randint(2, size=(N, 19, 19, 2))
end = end.astype ('float32')
groups = np.zeros((N, 19, 19, 1))
groups = groups.astype ('float32')
print ("getValidation", flush = True)
golois.getValidation (input_data, policy, value, end)
golois.getValidation (input_data, policy, value, end)

# Load models
h5_files = []
for name in os.listdir():
  if name[-2:] == 'h5':
    h5_files.append(name)

# if only one model
h5_files = ['model_resnet_Adam_lr1_10-4.h5']
for model_filename in h5_files:
  model = keras.models.load_model(model_filename)
  m_names = model.metrics_names

  val = model.evaluate (input_data,
                        [policy, value], verbose = 0, batch_size=batch)
  print("")
  print("*******************")
  print(model_filename)
  for i, metric in enumerate(m_names):
    print(metric, ":", val[i])
  #display(keras.utils.plot_model(model, str(model_name + ".png"), show_shapes=True))

getValidation

*******************
model_resnet_Adam_lr1_10-4.h5
loss : 3.718195676803589
policy_loss : 3.0275330543518066
value_loss : 0.6893633008003235
policy_categorical_accuracy : 0.3059000074863434
value_mse : 0.11816979944705963
